In [ ]:
!pip install nbformat nbconvert requests


In [ ]:
!pip install nbformat nbconvert requests


In [ ]:
import webbrowser

def open_notebook(option):
    links = {
        1: "https://raw.githubusercontent.com/sokrypton/ColabFold/main/AlphaFold2.ipynb",
        2: "https://colab.research.google.com/drive/18YUnY1EhtDcRuwrV5cTIAhYkK-NwBYym",
        3: "https://colab.research.google.com/github/deepmind/graphcast/blob/master/graphcast_demo.ipynb#scrollTo=J4FJFKWD8Loz",
    }

    link = links.get(option)
    if link:
        webbrowser.open(link)
        print(f"Opening Notebook {option}: {link}")
    else:
        print("Invalid option. Please choose 1, 2, or 3.")

def main():
    while True:
        print("\nChoose a Notebook to open:")
        print("1: AlphaFold2 Notebook")
        print("2: Colab Notebook from Drive")
        print("3: GraphCast Demo Notebook")
        print("0: Exit")

        try:
            choice = int(input("Enter your choice (1/2/3 or 0 to exit): "))
            if choice == 0:
                print("Exiting...")
                break
            open_notebook(choice)
        except ValueError:
            print("Invalid input. Please enter a number.")

if __name__ == "__main__":
    main()



Choose a Notebook to open:
1: AlphaFold2 Notebook
2: Colab Notebook from Drive
3: GraphCast Demo Notebook
0: Exit
Enter your choice (1/2/3 or 0 to exit): 2
Opening Notebook 2: https://colab.research.google.com/drive/18YUnY1EhtDcRuwrV5cTIAhYkK-NwBYym

Choose a Notebook to open:
1: AlphaFold2 Notebook
2: Colab Notebook from Drive
3: GraphCast Demo Notebook
0: Exit
Enter your choice (1/2/3 or 0 to exit): 2
Opening Notebook 2: https://colab.research.google.com/drive/18YUnY1EhtDcRuwrV5cTIAhYkK-NwBYym

Choose a Notebook to open:
1: AlphaFold2 Notebook
2: Colab Notebook from Drive
3: GraphCast Demo Notebook
0: Exit


KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install nbformat papermill


In [1]:
import nbformat
import requests
import os
from nbformat.v4 import new_notebook, new_code_cell, new_markdown_cell
from IPython.core.interactiveshell import InteractiveShell
from google.colab import files
import warnings

# Install missing dependencies
!pip install biopython
!pip install py3Dmol

# Function to download the notebook from a URL or take the uploaded filename
def download_notebook(link):
    """
    Downloads a notebook from the given URL or takes a user-provided filename.
    Returns the name of the saved notebook.
    """
    if "colab.research.google.com" in link:
        # For Colab links, manual download is required
        print(f"Please manually download the notebook from the link and save it locally.")
        notebook_name = input("Enter the name of the notebook file you saved (including .ipynb extension): ")
        if not os.path.exists(notebook_name):
            raise FileNotFoundError(f"File '{notebook_name}' not found. Ensure you downloaded and saved it correctly.")
    elif link.endswith(".ipynb"):
        # Extract filename from the URL
        notebook_name = link.split("/")[-1]
        print(f"Downloading notebook from: {link}")
        response = requests.get(link)
        response.raise_for_status()
        with open(notebook_name, 'w') as f:
            f.write(response.text)
        print(f"Downloaded notebook saved as: {notebook_name}")
    else:
        raise ValueError("Invalid link format. Must be a raw .ipynb file or a Colab notebook link.")

    return notebook_name

# Function to add kernel metadata to the notebook (ensures compatibility with different environments)
def add_kernel_metadata(notebook_path, kernel_name="python3"):
    with open(notebook_path, 'r') as f:
        nb = nbformat.read(f, as_version=4)

    if 'kernelspec' not in nb.metadata:
        nb.metadata['kernelspec'] = {
            "display_name": "Python 3",
            "language": "python",
            "name": kernel_name
        }
        print(f"Added kernel metadata to {notebook_path}.")

    with open(notebook_path, 'w') as f:
        nbformat.write(nb, f)

# Function to execute a single cell in the notebook and append the output
def execute_cell(cell_code, notebook, shell):
    """
    Executes a single cell's code and appends the output to the notebook.
    """
    try:
        result = shell.run_cell(cell_code)
        # Capture output and append to notebook
        output = result.stdout + result.stderr
        cell = new_code_cell(cell_code)
        cell.outputs = [{
            'output_type': 'stream',
            'name': 'stdout',
            'text': output
        }]
        notebook.cells.append(cell)
    except Exception as e:
        # Append the error message to notebook as markdown cell
        error_cell = new_markdown_cell(f"**Error:** {str(e)}")
        notebook.cells.append(error_cell)

# Function to execute the entire notebook and update outputs
def execute_notebook(notebook_path, output_path):
    """
    Execute each cell in the notebook and update the notebook with results.
    """
    # Load the notebook
    with open(notebook_path, 'r') as f:
        nb = nbformat.read(f, as_version=4)

    # Prepare the interactive shell to execute code
    shell = InteractiveShell.instance()

    # Create a new notebook to store the results
    output_notebook = new_notebook(cells=[])

    # Iterate through cells and execute them
    for cell in nb.cells:
        if cell.cell_type == 'code':
            cell_code = cell.source
            execute_cell(cell_code, output_notebook, shell)
        else:
            # Non-code cells (like markdown) just copy over
            output_notebook.cells.append(cell)

    # Save the executed notebook with outputs
    with open(output_path, 'w') as f:
        nbformat.write(output_notebook, f)
    print(f"Executed notebook saved as: {output_path}")

# Cleanup temporary files after execution
def cleanup(files):
    for file in files:
        try:
            os.remove(file)
            print(f"Deleted temporary file: {file}")
        except OSError as e:
            print(f"Error deleting file {file}: {e}")

# Main function that interacts with the user
def main():
    # List of notebooks (you can expand this list)
    notebook_links = []

    # Ask the user to input the URLs for the notebooks
    num_notebooks = int(input("How many notebooks would you like to provide links for? "))
    for i in range(num_notebooks):
        link = input(f"Please provide the URL for notebook {i+1}: ")
        notebook_links.append(link)

    # Ask user to choose which notebook to execute
    print("\nPlease select which notebook you would like to execute:")
    for i, link in enumerate(notebook_links, start=1):
        print(f"{i}. {link}")

    choice = int(input("\nEnter the number of the notebook to execute: "))
    if choice < 1 or choice > len(notebook_links):
        print("Invalid choice.")
        return

    chosen_link = notebook_links[choice - 1]

    # Step 1: Download the selected notebook
    try:
        notebook_filename = download_notebook(chosen_link)
        add_kernel_metadata(notebook_filename)
    except Exception as e:
        print(f"Error downloading the selected notebook: {e}")
        return

    # Step 2: Execute the selected notebook
    executed_notebook_path = "executed_" + notebook_filename
    try:
        execute_notebook(notebook_filename, executed_notebook_path)
    except Exception as e:
        print(f"Error executing the notebook: {e}")
        return

    # Step 3: Ask if the user wants to download the results (if applicable)
    download_result = input("\nDo you want to download the result file (e.g., .zip)? (y/n): ")
    if download_result.lower() == 'y':
        result_file = input("Please provide the result file name (e.g., test_a5e17_3.result.zip): ")
        if os.path.exists(result_file):
            files.download(result_file)
        else:
            print(f"Result file {result_file} not found.")

    # Step 4: Cleanup temporary files
    cleanup([notebook_filename])

if __name__ == "__main__":
    main()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 41.4 MB/s eta 0:00:00
How many notebooks would you like to provide links for? 34
Please provide the URL for notebook 1: https://colab.research.google.com/drive/1Fo8_WjTyoUoeJDK5iidJs_7A1C1YcpKb
Please provide the URL for notebook 2: https://colab.research.google.com/drive/15eI8JUikadf6OkFO38h8fijIcBL6fnvq
Please provide the URL for notebook 3: https://colab.research.google.com/drive/1jA3w8AMTZhARXqtCqvTW5m3-j3xfDWnH
Please provide the URL for notebook 4: https://colab.research.google.com/drive/1VVwQ8HVJxeMf_i3FSOUu0yKTGfIlRmle
Please provide the URL for notebook 5: https://colab.research.google.com/drive/1Xd04UIzLWiQ7HUHTAmncmlLE-KHdGxjg
Please provide the URL for notebook 6: https://colab.research.google.com/drive/1Sceh5EBLfnP7w1fhgUSJkPg9e9W_bEMH
Please provide the URL for notebook 7: https://colab.research.google.com/drive/15fh0qnUxKfE6zIPqFYRFAhxwYMIaI6Bm
Please provide the URL for notebook 8: https://colab.research.google.com/

KeyboardInterrupt: Interrupted by user